In [ ]:
import pandas as pd
import numpy as np
import statistics
from datetime import date
from datetime import timedelta
from time import process_time 

In [ ]:
import Graph
import Path
import Evolutionary
import Tabu

# Testing graph implementation

In [ ]:
graph = Graph.get_graph()
assert len(graph.edges) == len(graph.vertices)

assert graph.get_neighbours('Czechia') == graph.get_neighbours('Czech_Republic')

assert np.array_equal(graph.get_stats('Czechia'), graph.get_stats('Czech_Republic'))

assert graph.get_active_cases("ABKHAZIA", date(2020, 3, 10)) == 0

assert graph.get_active_cases("Poland", date(2020, 3, 10)) == 17

assert graph.get_active_cases("Poland", date(2020, 3, 16)) == 122

assert graph.get_active_cases("Poland", date(2020, 3, 15)) == 102

assert graph.get_active_cases('POLAND', date(2019, 3, 10)) == 0

assert graph.get_active_cases('POLAND', date(2020, 3, 11)) - graph.get_active_cases('POLAND', date(2020, 3, 10)) == 5

# Testing path

In [ ]:
path = Path.Path('Poland', date(2020, 5, 1))

path.add('Czechia')
path.add('Germany')

assert path.countries == ['Poland', 'Czechia', 'Germany']

path.eval(graph)
score = graph.get_active_cases('Poland', date(2020, 5, 1)) + graph.get_active_cases(
    'Czechia', date(2020, 5, 2)) + graph.get_active_cases('Germany', date(2020, 5, 3))
assert score == path.score

path2 = Path.Path('Poland', date(2020, 5, 1))
path2.eval(graph)

assert path2.countries == ['Poland']

assert path2.score == 12253

assert graph.get_active_cases('Poland', date(2020, 5, 1)) == 12253

assert graph.get_active_cases('Czechia', date(2020, 5, 2)) == 7501

assert path2.get_profit(graph, 'Czechia') == 7501

# Testing evolutionary algorithm

In [ ]:
start_day = date(2020, 4, 24)
test = Evolutionary.EvolutionaryAlgorithm(graph, start_day, 100, 120, 0.05, 'roulette', 50, 'recovered')

best = test.find_path()

t = test.reproduce(test.random_lambda())

y = test.select_next_generation(t)

assert len(y) == len(test.active_generation)

In [ ]:
#prototype of method used for paths crossing
def cross_paths(path1, path2):
    itrsct = [value for value in path1 if value in path2]
    point1, point2 = path1.index(itrsct[len(itrsct)//2]), path2.index(itrsct[len(itrsct)//2])
    need_len = len(path2)
    if(point1 == point2):
        ch1 = path1[:point1] + path2[point2:]
        ch2 = path2[:point2] + path1[point1:]
        return ch1, ch2
    if(point1 < point2):
        tmp, tmpp = path1, point1
        path1, point1 = path2, point2
        path2, point2 = tmp, tmpp
    path = path1[:point1] + path2[point2:]
    ch1 = path[:need_len]
    ch2 = path[-need_len:]
    return ch1, ch2
    

path1 = ['q', 'w', 'e', '[]', 'r', 't', 'y', 'u', 'i', 'o']
path2 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']

assert cross_paths(path1, path2) == (['q', 'w', 'e', '[]', 'f', 'g', 'h', 'j', 'k', 'l'],
 ['a', 's', 'd', '[]', 'r', 't', 'y', 'u', 'i', 'o'])

path1 = ['q', 'w', 'e', 'r', 't', 'y', '[]', 'u', 'i', 'o']
path2 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']

assert cross_paths(path1, path2) == (['q', 'w', 'e', 'r', 't', 'y', '[]', 'f', 'g', 'h'],
 ['r', 't', 'y', '[]', 'f', 'g', 'h', 'j', 'k', 'l'])

path1 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']
path2 = ['q', 'w', 'e', 'r', 't', 'y', '[]', 'u', 'i', 'o']

assert cross_paths(path1, path2) == (['q', 'w', 'e', 'r', 't', 'y', '[]', 'f', 'g', 'h'],
 ['r', 't', 'y', '[]', 'f', 'g', 'h', 'j', 'k', 'l'])

path1 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']
path2 = ['q', 'w', '[]', 'e', 'r', 't', 'y', 'u', 'i', 'o']

assert cross_paths(path1, path2) == (['a', 's', 'd', '[]', 'e', 'r', 't', 'y', 'u', 'i'],
 ['s', 'd', '[]', 'e', 'r', 't', 'y', 'u', 'i', 'o'])

path1 = ['a', '[]', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l']
path2 = ['q', 'w', 'e', 'r', 't', '[]', 'y', 'u', 'i', 'o']

assert cross_paths(path1, path2) == (['q', 'w', 'e', 'r', 't', '[]', 's', 'd', 'f', 'g'],
 ['t', '[]', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l'])

path2 = ['a', 's', 'd', '[]', 'f', 'g', 'h', 'j', 'k', 'l']
path1 = ['q', 'w', 'e', 'r', 't', 'y', 'u', '[]', 'i', 'o']

assert cross_paths(path1, path2) == (['q', 'w', 'e', 'r', 't', 'y', 'u', '[]', 'f', 'g'],
 ['t', 'y', 'u', '[]', 'f', 'g', 'h', 'j', 'k', 'l'])

# Measuring Evolutionary algorithm performance

In [ ]:
start_day = date.today() - timedelta(days = 14)

In [ ]:
f = open('tests_log_mut.txt', 'w')

In [ ]:
repeats = 20
selections = ['ranking']
mutations = [0.1, 0.25, 0.5, 0.75, 0.9]
ni_list = [20]
generations = [60]

f.write('LOGS FROM EVOLUTIONARY TESTING 14 days score and time date'+date.today().strftime("%d/%m/%Y")+'\n\n')
for selection in selections:
    f.write('SELECTION METHOD: ' + selection + '\n')
    for mutation in mutations:
        f.write('MUTATION PARAM: '+str(mutation)+'\n')
        for ni in ni_list:
            f.write('NI PARAM: '+str(ni)+'\n')
            for t in generations:
                f.write('GENERATIONS PARAM: '+str(t)+'\n')
                scores = []
                times = []
                for i in range(repeats):
                    test = Evolutionary.EvolutionaryAlgorithm(graph, 
                                                          start_day, 
                                                          ni, 
                                                          int(1.2*ni), 
                                                          mutation, 
                                                          selection, 
                                                          t, 'recovered')
                    startt = process_time() 
                    best = test.find_path()
                    stopt = process_time()
                    times.append(stopt - startt)
                    scores.append(best.score)
                mean_score = statistics.mean(scores)
                mean_time = statistics.mean(times)
                f.write(str(mean_score)+'    '+str(mean_time)+'s\n')
                print(str(mean_score)+'    '+str(mean_time)+'s\n')
        

In [ ]:
f.close()

# Measuring tabu search algorithm performance

In [ ]:
f = open('tests_tabu_log.txt', 'w')

In [ ]:
tabu_lens = [10, 50, 100]
iters = [20, 50, 100, 200, 500] #, 1000, 5000, 10000]
repeats = 20

f.write('LOGS FROM TABU_SEARCH TESTING 14 days score and time date'+date.today().strftime("%d/%m/%Y")+'\n\n')
for tabu_len in tabu_lens:
    f.write('TABU LEN: ' + str(tabu_len) + '\n')
    for itera in iters:
        f.write('ITERS: '+str(itera)+'\n')
        scores = []
        times = []
        for i in range(repeats):
            test = Tabu.TabuSearch(graph, start_day, tabu_len, 'recovered')
            
            startt = process_time() 
            best = test.find_path(itera)
            stopt = process_time()
            
            times.append(stopt - startt)
            scores.append(best.score)
            
        mean_score = statistics.mean(scores)
        mean_time = statistics.mean(times)
        f.write(str(mean_score)+'    '+str(mean_time)+'s\n')
        print(str(mean_score) + '    '+str(mean_time)+'s\n')

In [ ]:
f.close()